In [12]:
import argparse
import json
from tqdm import tqdm
import datasets
import transformers
from transformers.integrations import TensorBoardCallback
from torch.utils.tensorboard import SummaryWriter
from transformers import TrainingArguments,TrainerCallback
from transformers import Trainer, HfArgumentParser
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType
from dataclasses import dataclass, field
import datasets
import os
from transformers import AutoTokenizer, AutoModel
import torch
from torch import nn

In [13]:
def preprocess(tokenizer, config, example, max_seq_length):
    #问题
    prompt = example["context"]
    #答案
    target = example["target"]
    #问题分词
    prompt_ids = tokenizer.encode(prompt, max_length=max_seq_length, truncation=True)
    #答案分词
    target_ids = tokenizer.encode(
        target,
        max_length=max_seq_length,
        truncation=True,
        add_special_tokens=False)
    input_ids = prompt_ids + target_ids + [config.eos_token_id]
    #input_ids:问题分词+答案分词  seq_len:问题长度
    return {"input_ids": input_ids, "seq_len": len(prompt_ids)}

In [14]:
model_dir = "/root/autodl-tmp/model/"
model_name = "chatglm"
model_path = os.path.join(model_dir,model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_path,trust_remote_code=True) #导入分词器
config = transformers.AutoConfig.from_pretrained(model_path, trust_remote_code=True, device_map='auto') #导入模型配置
def read_jsonl(path, max_seq_length, skip_overlength=False):
    with open(path, "r",encoding="utf-8") as f:
        for line in tqdm(f.readlines()):
            example = json.loads(line)
            feature = preprocess(tokenizer, config, example, max_seq_length)
            if skip_overlength and len(feature["input_ids"]) > max_seq_length:
                continue
            feature["input_ids"] = feature["input_ids"][:max_seq_length]
            yield feature

In [15]:
data_dir = "/root/autodl-tmp/data/wenlv/"
json_name = "wenlv.jsonl"
save_name = "wenlv_token"
json_path = os.path.join(data_dir,json_name)
save_path = os.path.join(data_dir,save_name)
max_seq_length = 300
skip_overlength = False
dataset = datasets.Dataset.from_generator(lambda: read_jsonl(json_path, max_seq_length, skip_overlength))
dataset.save_to_disk(save_path)

Saving the dataset (0/1 shards):   0%|          | 0/2956 [00:00<?, ? examples/s]